# 필수라이브러리 및 데이터 경로 정적으로 정의 밑 import

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
DATA_PATH='/kaggle/input/abstract-data-set-for-credit-card-fraud-detection/creditcardcsvpresent.csv'

In [ ]:
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
entire_df = pd.read_csv(DATA_PATH)

정규화 함수들 정의함

In [ ]:
def regularization_l2(data_arr):
    return (data_arr/ np.linalg.norm(data_arr))

In [ ]:
def regularization_mean_std_d(data_arr):
    mean = np.mean(data_arr)
    std = np.std(data_arr) 
    regularaz_arr = (data_arr - mean)/std
    return regularaz_arr

In [ ]:
def sub_mean(data_arr):
    mean = np.mean(data_arr)
    regularaz_arr = (data_arr - mean)
    return regularaz_arr

hw01의 3번 과제에서 영감을 얻어서 한번 시도해봄

In [ ]:
def max_min_reg(data_arr):
    max_ = np.max(data_arr)
    min_ = np.min(data_arr) 
    regularaz_arr = (data_arr - min_)/(max_-min_)
    return regularaz_arr

hw01의 3번 과제에서 나쁘지 않은 성능을 발휘해서 도움이 될수도 잇을거같아서 가져옴

In [ ]:
entire_df.head()

In [ ]:
entire_df.describe()

솔직히 그냥봐도 Merchant_id, Transaction date는 못써먹게 생겼습니다만 혹시나 모르니 한번 Transaction date를 조사해보겠습니다.

In [ ]:
entire_df.isnull().sum()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))



entire_df['isFradulent'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot - isFradulent')
ax[0].set_ylabel('')
sns.countplot('isFradulent', data=entire_df, ax=ax[1])
ax[1].set_title('Count plot - isFradulent')

plt.show()

사기인가 아닌가 레이블 비교인데, 여기서는 Y의 비율이 15프로로 매우 적으므로 accuracy를 계산하는것은 의미가 없고 TPR기준은 f1 스코어를 사용하여서 스코어링 합니다.

In [ ]:
dropcolums= ['Merchant_id','Transaction date']

Merchant_id랑 Transaction date는 쓸모 없는 데이터 이므로 바로 제거 해줍니다.

In [ ]:
entire_df = entire_df.drop(dropcolums,axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(entire_df,test_size=0.25, random_state=2020)

이진분류데이터인데 레이블을 Y는 1로 N은 0으로 바꿔줍니다.

In [ ]:
df_train['isFradulent']= df_train['isFradulent'].replace('N',0)
df_train['isFradulent']=df_train['isFradulent'].replace('Y',1)

In [ ]:
df_test['isFradulent']= df_test['isFradulent'].replace('N',0)
df_test['isFradulent']=df_test['isFradulent'].replace('Y',1)

<h2>Average Amount/transaction/day</h2>

In [ ]:
df_train['Average Amount/transaction/day'].describe()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(df_train[df_train['isFradulent']==0]['Average Amount/transaction/day'],ax=ax)
sns.kdeplot(df_train[df_train['isFradulent']==1]['Average Amount/transaction/day'],ax=ax)
plt.legend(['isFradulent==0','isFradulent==1'])
plt.title('Ogriginal Average Amount/transaction/day distributution')
plt.show()

In [ ]:
plt.hist(df_train[df_train['isFradulent']==0]['Average Amount/transaction/day'])
plt.hist(df_train[df_train['isFradulent']==1]['Average Amount/transaction/day'])
plt.legend(['isFradulent==0','isFradulent==1'])
plt.show()

그래프랑 히스토그램을 봤을 때, 딱봐도 이상치가 있는게 보입니다.

# Remove Outlier

통계학에서 자주 사용하는 75프로 대 -25프로 대 *3을 기준점으로 이거 선넘으면 이상치로 간주하고 제거하는 정석적인 방법을 사용하여 2000 근처에 있는 녀석들을 제거해줍니다.

In [ ]:
per_df_tr_of_75 = df_train['Average Amount/transaction/day'].quantile(q=0.75, interpolation='nearest')
per_df_tr_of_25 = df_train['Average Amount/transaction/day'].quantile(q=0.25, interpolation='nearest')
out_line = (per_df_tr_of_75-per_df_tr_of_25)*3
df_train = df_train.drop(df_train.loc[df_train['Average Amount/transaction/day']>out_line].index.values.astype(int))

In [ ]:
plt.hist(df_train[df_train['isFradulent']==0]['Average Amount/transaction/day'])
plt.hist(df_train[df_train['isFradulent']==1]['Average Amount/transaction/day'])
plt.legend(['isFradulent==0','isFradulent==1'])
plt.show()

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(9,5))

sns.kdeplot(np.log(df_train[df_train['isFradulent']==0]['Average Amount/transaction/day']),ax = ax[0][0])
sns.kdeplot(np.log(df_train[df_train['isFradulent']==1]['Average Amount/transaction/day']),ax=ax[0][0])
ax[0][0].legend(['isFradulent==0','isFradulent==1'])
ax[0][0].set_title('Log')

sns.kdeplot(np.sin(df_train[df_train['isFradulent']==0]['Average Amount/transaction/day']),ax = ax[0][1])
sns.kdeplot(np.sin(df_train[df_train['isFradulent']==1]['Average Amount/transaction/day']),ax=ax[0][1])
ax[0][1].legend(['target==0','target==1'])
ax[0][1].set_title('Sin ')

sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==0]['Average Amount/transaction/day']),ax = ax[0][2])
sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==1]['Average Amount/transaction/day']),ax=ax[0][2])
ax[0][2].legend(['target==0','target==1'])
ax[0][2].set_title('sub_mean ')

sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==0]['Average Amount/transaction/day']),ax = ax[1][0])
sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==1]['Average Amount/transaction/day']),ax=ax[1][0])
ax[1][0].legend(['target==0','target==1'])
ax[1][0].set_title('sub_mean_std_reg ')

sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==0]['Average Amount/transaction/day']),ax = ax[1][1])
sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==1]['Average Amount/transaction/day']),ax=ax[1][1])
ax[1][1].legend(['target==0','target==1'])
ax[1][1].set_title('l2 reg ')

sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==0]['Average Amount/transaction/day']),ax = ax[1][2])
sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==1]['Average Amount/transaction/day']),ax=ax[1][2])
ax[1][2].legend(['target==0','target==1'])
ax[1][2].set_title('min_max reg')
plt.show()

여기서 저는 l2를 선택하겠습니다. 

<h2>Transaction_amount </h2>

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(df_train[df_train['isFradulent']==0]['Transaction_amount'],ax = ax)
sns.kdeplot(df_train[df_train['isFradulent']==1]['Transaction_amount'],ax=ax)
plt.legend(['isFradulent==0','isFradulent==1'])
plt.title('Transaction_amount distributution')
plt.show()

In [ ]:
plt.hist(df_train[df_train['isFradulent']==0]['Transaction_amount'])
plt.hist(df_train[df_train['isFradulent']==1]['Transaction_amount'])
plt.legend(['isFradulent==0','isFradulent==1'])
plt.show()

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(9,5))

sns.kdeplot(np.log(df_train[df_train['isFradulent']==0]['Transaction_amount']),ax = ax[0][0])
sns.kdeplot(np.log(df_train[df_train['isFradulent']==1]['Transaction_amount']),ax=ax[0][0])
ax[0][0].legend(['isFradulent==0','isFradulent==1'])
ax[0][0].set_title('Log')

sns.kdeplot(np.sin(df_train[df_train['isFradulent']==0]['Transaction_amount']),ax = ax[0][1])
sns.kdeplot(np.sin(df_train[df_train['isFradulent']==1]['Transaction_amount']),ax=ax[0][1])
ax[0][1].legend(['target==0','target==1'])
ax[0][1].set_title('Sin ')

sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==0]['Transaction_amount']),ax = ax[0][2])
sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==1]['Transaction_amount']),ax=ax[0][2])
ax[0][2].legend(['target==0','target==1'])
ax[0][2].set_title('sub_mean ')

sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==0]['Transaction_amount']),ax = ax[1][0])
sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==1]['Transaction_amount']),ax=ax[1][0])
ax[1][0].legend(['target==0','target==1'])
ax[1][0].set_title('sub_mean_std_reg ')

sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==0]['Transaction_amount']),ax = ax[1][1])
sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==1]['Transaction_amount']),ax=ax[1][1])
ax[1][1].legend(['target==0','target==1'])
ax[1][1].set_title('l2 reg ')

sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==0]['Transaction_amount']),ax = ax[1][2])
sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==1]['Transaction_amount']),ax=ax[1][2])
ax[1][2].legend(['target==0','target==1'])
ax[1][2].set_title('min_max reg')
plt.show()

여기서도 l2가 전처리 전의 특성을 가장 잘 살린거 같아서 l2를 사용하겠습니다.

<h2>Is declined,  Total Number of declines/day</h2>

In [ ]:
df_train['Is declined'].describe()

declined을 햇냐 안했냐의 2진 데이터 입니다. 아래의 Total Number of declines/day와 뭔가 관련이 잇어보이기도 하는군요

In [ ]:
df_train['Total Number of declines/day'].describe()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(9,5))
sns.countplot(df_train[df_train['isFradulent']==0]['Total Number of declines/day'],ax =ax[0])
sns.countplot(df_train[df_train['isFradulent']==1]['Total Number of declines/day'],ax=ax[0])
ax[0].legend(['isFradulent==0','isFradulent==1'])


ax[1].hist(df_train[df_train['isFradulent']==0]['Total Number of declines/day'])
ax[1].hist(df_train[df_train['isFradulent']==1]['Total Number of declines/day'])
ax[1].legend(['isFradulent==0','isFradulent==1'])
plt.show()

Total Number of declines/day의 사기와 사기 아닌거의 분포 차이입니다.decline이 많을 수록 사기일 확률이 높네요 

In [ ]:
sns.violinplot('Total Number of declines/day','Is declined',hue='isFradulent',data=df_train,scale='count')


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(df_train[df_train['isFradulent']==0]['Total Number of declines/day'],ax = ax,cumulative=True, bw=1.5)
sns.kdeplot(df_train[df_train['isFradulent']==1]['Total Number of declines/day'],ax=ax,cumulative=True, bw=1.5)
plt.legend(['isFradulent==0','isFradulent==1'])
plt.title('Transaction_amount distributution')
plt.show()

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(9,5))

sns.kdeplot(np.log(df_train[df_train['isFradulent']==0]['Total Number of declines/day']),ax = ax[0][0],cumulative=True, bw=1.5)
sns.kdeplot(np.log(df_train[df_train['isFradulent']==1]['Total Number of declines/day']),ax=ax[0][0],cumulative=True, bw=1.5)
ax[0][0].legend(['isFradulent==0','isFradulent==1'])
ax[0][0].set_title('Log')

sns.kdeplot(np.sin(df_train[df_train['isFradulent']==0]['Total Number of declines/day']),ax = ax[0][1],cumulative=True, bw=1.5)
sns.kdeplot(np.sin(df_train[df_train['isFradulent']==1]['Total Number of declines/day']),ax=ax[0][1],cumulative=True, bw=1.5)
ax[0][1].legend(['target==0','target==1'])
ax[0][1].set_title('Sin ')

sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==0]['Total Number of declines/day']),ax = ax[0][2],cumulative=True, bw=1.5)
sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==1]['Total Number of declines/day']),ax=ax[0][2],cumulative=True, bw=1.5)
ax[0][2].legend(['target==0','target==1'])
ax[0][2].set_title('sub_mean ')

sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==0]['Total Number of declines/day']),ax = ax[1][0],cumulative=True, bw=1.5)
sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==1]['Total Number of declines/day']),ax=ax[1][0],cumulative=True, bw=1.5)
ax[1][0].legend(['target==0','target==1'])
ax[1][0].set_title('sub_mean_std_reg ')

sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==0]['Total Number of declines/day']),ax = ax[1][1],cumulative=True, bw=1.5)
sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==1]['Total Number of declines/day']),ax=ax[1][1],cumulative=True, bw=1.5)
ax[1][1].legend(['target==0','target==1'])
ax[1][1].set_title('l2 reg ')

sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==0]['Total Number of declines/day']),ax = ax[1][2],cumulative=True, bw=1.5)
sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==1]['Total Number of declines/day']),ax=ax[1][2],cumulative=True, bw=1.5)
ax[1][2].legend(['target==0','target==1'])
ax[1][2].set_title('min_max reg')
plt.show()

가장 log는 추후 사용될 전처리에서 -무한대에 수렴해버리는 일이 발생하여 제외시켜버리고 나머지는 다 고만고만하니 Y와 N의 차이가 그나마 좀 보이는 sub_mean을 선택하겠습니다.

In [ ]:
df_train['Total Number of declines/day'] = sub_mean(df_train['Total Number of declines/day'])

In [ ]:
df_test['Total Number of declines/day'] = sub_mean(df_test['Total Number of declines/day'])

In [ ]:
df_train['Is declined']= df_train['Is declined'].replace('N',-1)
df_train['Is declined']=df_train['Is declined'].replace('Y',1)

In [ ]:
df_test['Is declined']= df_test['Is declined'].replace('N',-1)
df_test['Is declined']=df_test['Is declined'].replace('Y',1)

이번엔 기존의 이중 피쳐에 대해 조금 다른 방식으로 접근해보았는데요,위의 hist그램에서 decline이 1인경우가 사기가많고 0이면, 상대적으로 사기가 적은 분포 특성을 이용하여, decline이 아니면 -1을 줘서 계산시 0으로 가깝게 1을 줘서 1을 최대한 유지하는 방식으로 서로 대칭되게 하는 실험적인 방법으로 접근 해보았습니다.

In [ ]:
df_train['Is declined'] = regularization_mean_std_d(df_train['Total Number of declines/day']) +df_train['Is declined']

In [ ]:
df_test['Is declined'] = regularization_mean_std_d(df_test['Total Number of declines/day']) +df_test['Is declined']

원래 Is declined피쳐는 이진 특성을 가졌지만 저는 -1과 1 의 대칭적인 성향을 주었으니 그에 따라 높은 상관성을 지니는 Total Number of declines/day의 정규화 값을 더해서 N이더라도 약간의 차이가 있도록 조정 해보았습니다.

In [ ]:
plt.hist(df_train[df_train['isFradulent']==0]['Is declined'])
plt.hist(df_train[df_train['isFradulent']==1]['Is declined'])
plt.legend(['isFradulent==0','isFradulent==1'])
plt.show()

In [ ]:
df_train[df_train['isFradulent']==0]['Is declined'].describe()

In [ ]:
df_train[df_train['isFradulent']==1]['Is declined'].describe()

<h2>isForeignTransaction </h2>

In [ ]:
df_train['isForeignTransaction'].describe()

In [ ]:
plt.hist(df_train[df_train['isFradulent']==0]['isForeignTransaction'])
plt.hist(df_train[df_train['isFradulent']==1]['isForeignTransaction'])
plt.legend(['isFradulent==0','isFradulent==1'])
plt.show()

이역시도 2진분류인데 외국인이 상대적으로 사기꾼이 많은 것을 알수 있는 차트입니다.

In [ ]:
pd.crosstab(df_train['isForeignTransaction'], df_train['isFradulent'], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
df_train['isHighRiskCountry'].describe()

# isHighRiskCountry

In [ ]:
plt.hist(df_train[df_train['isFradulent']==0]['isHighRiskCountry'])
plt.hist(df_train[df_train['isFradulent']==1]['isHighRiskCountry'])
plt.legend(['isFradulent==0','isFradulent==1'])
plt.show()

In [ ]:
pd.crosstab(df_train['isHighRiskCountry'], df_train['isFradulent'], margins=True).style.background_gradient(cmap='summer_r')

고위험 국가인가에 대한 차트인데 의외로 사기의 비율은 동일했습니다. 확실한건 고 위험 국가에서는 사기를 안치는 경우는 거의 없군요

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,7))
sns.violinplot(x='isForeignTransaction',y='isFradulent',data=df_train,split=True,ax=ax[0])
ax[0].set_yticks(range(0,3,1))
ax[0].set_title('isForeignTransaction with isFradulent')
sns.violinplot(x='isHighRiskCountry',y='isFradulent',data=df_train,split=True,ax=ax[1])
ax[1].set_yticks(range(0,3,1))
ax[1].set_title('isHighRiskCountry with isFradulent')
plt.show()

In [ ]:
df_train['isHighRiskCountry']= df_train['isHighRiskCountry'].replace('N',-1)
df_train['isHighRiskCountry']=df_train['isHighRiskCountry'].replace('Y',1)

df_train['isForeignTransaction']= df_train['isForeignTransaction'].replace('N',1)
df_train['isForeignTransaction']=df_train['isForeignTransaction'].replace('Y',0)


In [ ]:
df_test['isHighRiskCountry']= df_test['isHighRiskCountry'].replace('N',-1)
df_test['isHighRiskCountry']=df_test['isHighRiskCountry'].replace('Y',1)

df_test['isForeignTransaction']= df_test['isForeignTransaction'].replace('N',1)
df_test['isForeignTransaction']=df_test['isForeignTransaction'].replace('Y',0)


isForeignTransaction, isHighRiskCountry 이 두개의 피쳐 특성상 둘다 극단적으로 몰려있기는한데, 위의 Is Declined처럼 같이 엮어서 처리할수 잇는 범주형데이터가 없으므로 고 위험 국가일경우 사기를 치는 경우가 많은 isHighRiskCountry에서는 N Y를 -1 1로 isForeignTransaction은 0 1로 바꿔주고 진행합니다. 지나친 전처리는 다중 공선성으로인해 오버피팅 될수 있어서 이정도 선에서 끝냅니다.

<h2>Daily_chargeback_avg_amt</h2>

In [ ]:
df_train['Daily_chargeback_avg_amt'].describe()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(df_train[df_train['isFradulent']==0]['Daily_chargeback_avg_amt'],ax=ax,cumulative=True, bw=1.5)
sns.kdeplot(df_train[df_train['isFradulent']==1]['Daily_chargeback_avg_amt'],ax=ax,cumulative=True, bw=1.5)
plt.legend(['isFradulent==0','isFradulent==1'])
plt.title('Daily_chargeback_avg_amt distributution')
plt.show()

In [ ]:
plt.hist(df_train[df_train['isFradulent']==0]['Daily_chargeback_avg_amt'])
plt.hist(df_train[df_train['isFradulent']==1]['Daily_chargeback_avg_amt'])
plt.legend(['isFradulent==0','isFradulent==1'])
plt.show()

In [ ]:
pd.crosstab(df_train['Daily_chargeback_avg_amt'], df_train['isFradulent'], margins=True).style.background_gradient(cmap='summer_r')

중간중간에 노이즈가 있는데, 데이터가 너무 극단적으로 좌로 치우쳐져있어서 그리고 중간에 피쳐들이 몰려있는 구간이 있기 때문에, 우선 그대로 사용해보겠습니다.

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(9,5))

sns.kdeplot(np.log(df_train[df_train['isFradulent']==0]['Daily_chargeback_avg_amt']),ax = ax[0][0],cumulative=True, bw=1.5)
sns.kdeplot(np.log(df_train[df_train['isFradulent']==1]['Daily_chargeback_avg_amt']),ax=ax[0][0],cumulative=True, bw=1.5)
ax[0][0].legend(['isFradulent==0','isFradulent==1'])
ax[0][0].set_title('Log')

sns.kdeplot(np.sin(df_train[df_train['isFradulent']==0]['Daily_chargeback_avg_amt']),ax = ax[0][1],cumulative=True, bw=1.5)
sns.kdeplot(np.sin(df_train[df_train['isFradulent']==1]['Daily_chargeback_avg_amt']),ax=ax[0][1],cumulative=True, bw=1.5)
ax[0][1].legend(['target==0','target==1'])
ax[0][1].set_title('Sin ')

sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==0]['Daily_chargeback_avg_amt']),ax = ax[0][2],cumulative=True, bw=1.5)
sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==1]['Daily_chargeback_avg_amt']),ax=ax[0][2],cumulative=True, bw=1.5)
ax[0][2].legend(['target==0','target==1'])
ax[0][2].set_title('sub_mean ')

sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==0]['Daily_chargeback_avg_amt']),ax = ax[1][0],cumulative=True, bw=1.5)
sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==1]['Daily_chargeback_avg_amt']),ax=ax[1][0],cumulative=True, bw=1.5)
ax[1][0].legend(['target==0','target==1'])
ax[1][0].set_title('sub_mean_std_reg ')

sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==0]['Daily_chargeback_avg_amt']),ax = ax[1][1],cumulative=True, bw=1.5)
sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==1]['Daily_chargeback_avg_amt']),ax=ax[1][1],cumulative=True, bw=1.5)
ax[1][1].legend(['target==0','target==1'])
ax[1][1].set_title('l2 reg ')

sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==0]['Daily_chargeback_avg_amt']),ax = ax[1][2],cumulative=True, bw=1.5)
sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==1]['Daily_chargeback_avg_amt']),ax=ax[1][2],cumulative=True, bw=1.5)
ax[1][2].legend(['target==0','target==1'])
ax[1][2].set_title('min_max reg')
plt.show()

피쳐의 특성이 유지되면서 약간의 차이를 벌릴수있는 sub_mean을 사용하겠습니다.

In [ ]:
df_train['6_month_avg_chbk_amt'].describe()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(df_train[df_train['isFradulent']==0]['6_month_avg_chbk_amt'],ax=ax,cumulative=True, bw=1.5)
sns.kdeplot(df_train[df_train['isFradulent']==1]['6_month_avg_chbk_amt'],ax=ax,cumulative=True, bw=1.5)
plt.legend(['isFradulent==0','isFradulent==1'])
plt.title('6_month_avg_chbk_amt distributution')
plt.show()

In [ ]:
plt.hist(df_train[df_train['isFradulent']==0]['6_month_avg_chbk_amt'])
plt.hist(df_train[df_train['isFradulent']==1]['6_month_avg_chbk_amt'])
plt.legend(['isFradulent==0','isFradulent==1'])
plt.show()

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(9,5))

sns.kdeplot(np.log(df_train[df_train['isFradulent']==0]['6_month_avg_chbk_amt']),ax = ax[0][0],cumulative=True, bw=1.5)
sns.kdeplot(np.log(df_train[df_train['isFradulent']==1]['6_month_avg_chbk_amt']),ax=ax[0][0],cumulative=True, bw=1.5)
ax[0][0].legend(['isFradulent==0','isFradulent==1'])
ax[0][0].set_title('Log')

sns.kdeplot(np.sin(df_train[df_train['isFradulent']==0]['6_month_avg_chbk_amt']),ax = ax[0][1],cumulative=True, bw=1.5)
sns.kdeplot(np.sin(df_train[df_train['isFradulent']==1]['6_month_avg_chbk_amt']),ax=ax[0][1],cumulative=True, bw=1.5)
ax[0][1].legend(['target==0','target==1'])
ax[0][1].set_title('Sin ')

sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==0]['6_month_avg_chbk_amt']),ax = ax[0][2],cumulative=True, bw=1.5)
sns.kdeplot(sub_mean(df_train[df_train['isFradulent']==1]['6_month_avg_chbk_amt']),ax=ax[0][2],cumulative=True, bw=1.5)
ax[0][2].legend(['target==0','target==1'])
ax[0][2].set_title('sub_mean ')

sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==0]['6_month_avg_chbk_amt']),ax = ax[1][0],cumulative=True, bw=1.5)
sns.kdeplot(regularization_mean_std_d(df_train[df_train['isFradulent']==1]['6_month_avg_chbk_amt']),ax=ax[1][0],cumulative=True, bw=1.5)
ax[1][0].legend(['target==0','target==1'])
ax[1][0].set_title('sub_mean_std_reg ')

sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==0]['6_month_avg_chbk_amt']),ax = ax[1][1],cumulative=True, bw=1.5)
sns.kdeplot(regularization_l2(df_train[df_train['isFradulent']==1]['6_month_avg_chbk_amt']),ax=ax[1][1],cumulative=True, bw=1.5)
ax[1][1].legend(['target==0','target==1'])
ax[1][1].set_title('l2 reg ')

sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==0]['6_month_avg_chbk_amt']),ax = ax[1][2],cumulative=True, bw=1.5)
sns.kdeplot(max_min_reg(df_train[df_train['isFradulent']==1]['6_month_avg_chbk_amt']),ax=ax[1][2],cumulative=True, bw=1.5)
ax[1][2].legend(['target==0','target==1'])
ax[1][2].set_title('min_max reg')
plt.show()

In [ ]:
df_train['6-month_chbk_freq'].describe()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
sns.kdeplot(df_train[df_train['isFradulent']==0]['6-month_chbk_freq'],ax=ax,cumulative=True, bw=1.5)
sns.kdeplot(df_train[df_train['isFradulent']==1]['6-month_chbk_freq'],ax=ax,cumulative=True, bw=1.5)
plt.legend(['isFradulent==0','isFradulent==1'])
plt.title('6-month_chbk_freq distributution')
plt.show()

In [ ]:
plt.hist(df_train[df_train['isFradulent']==0]['6-month_chbk_freq'])
plt.hist(df_train[df_train['isFradulent']==1]['6-month_chbk_freq'])
plt.legend(['isFradulent==0','isFradulent==1'])
plt.show()

In [ ]:
sns.barplot(x="6-month_chbk_freq", y="6_month_avg_chbk_amt", data=df_train)

6-month_chbk_freq인데, 6개월 동안 몇번을 했는지에 대한 분포로 보입니다. 값 편차도 크지않아 아무것도 안건들겠습니다.

# 6_month_avg_chbk_amt

In [ ]:
df_train['6_month_avg_chbk_amt'].describe()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(9,5))
sns.kdeplot(df_train[df_train['isFradulent']==0]['6_month_avg_chbk_amt'],cumulative=True, bw=1.5,ax=ax[0])
sns.kdeplot(df_train[df_train['isFradulent']==1]['6_month_avg_chbk_amt'],cumulative=True, bw=1.5,ax=ax[0])
ax[0].legend(['isFradulent==0','isFradulent==1'])

ax[1].hist(df_train[df_train['isFradulent']==0]['6-month_chbk_freq'])
ax[1].hist(df_train[df_train['isFradulent']==1]['6-month_chbk_freq'])
ax[1].legend(['isFradulent==0','isFradulent==1'])
plt.show()

한쪽으로 쏠린 경향이 있는 데이터입니다만, 6-month_chbk_freq,6_month_avg_chbk_amt의 관계를 조금 더 살펴보고 어떻게 처리할지 결정하겠습니다.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

xs = df_train['6-month_chbk_freq']
ys = df_train['6_month_avg_chbk_amt']
zs = df_train['isFradulent']
ax.scatter(xs, ys, zs, s=50, alpha=0.6, edgecolors='w')

ax.set_xlabel('6-month_chbk_freq')
ax.set_ylabel('6_month_avg_chbk_amt')
ax.set_zlabel('isFradulent')
ax.scatter(xs, ys, zs, c = zs, s= 50, alpha=0.5, cmap=plt.cm.Greens)
#ax.view_init(20, 45)
plt.show()


3차원분포에서 알 수 있듯이 chbk freq가 적을 수록 사기일 가능성이 낮고, 그 횟수가 많을 수록 사기인것이 걸립니다. 하지만 노이즈 치곤 몰려있는 값들이 적지 않으므로, 
6_month_avg_chbk_amt에는 값 유지 및 수치 밸런스를 맞추기 위해 일반적인 정규화(평균으로 빼고 표준편차로 나누는거)만 하겠습니다.

<h2>Data Preprocessing</h2>

위에서 분석한 전략들로 전처리를 시행합니다.

In [ ]:
df_train['Average Amount/transaction/day'] = regularization_l2(df_train['Average Amount/transaction/day'])
df_train['Transaction_amount']= regularization_l2(df_train['Transaction_amount'])
df_train['Daily_chargeback_avg_amt']=sub_mean(df_train['Daily_chargeback_avg_amt'])
df_train['6_month_avg_chbk_amt']=regularization_mean_std_d(df_train['6_month_avg_chbk_amt'])

In [ ]:
df_test['Average Amount/transaction/day'] = regularization_l2(df_test['Average Amount/transaction/day'])
df_test['Transaction_amount']= regularization_l2(df_test['Transaction_amount'])
df_test['Daily_chargeback_avg_amt']=sub_mean(df_test['Daily_chargeback_avg_amt'])
df_test['6_month_avg_chbk_amt']=regularization_mean_std_d(df_test['6_month_avg_chbk_amt'])

# Train and Test

In [ ]:
y_train = np.asarray(df_train['isFradulent'])
x_train = np.asarray(df_train.drop(['isFradulent'],axis=1))

In [ ]:
y_test = np.asarray(df_test['isFradulent'])
x_test = np.asarray(df_test.drop(['isFradulent'],axis=1))

레이블을 분리해줍니다.

<h3>model import</h3>

In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [ ]:
model_performance_list = []
model_list = ['RandomForest','Xgboost']

In [ ]:
sf_rf_train_res_ensem = np.zeros(len(x_train))
sf_rf_test_res_ensem = np.zeros(len(x_test))
skfold = StratifiedKFold(n_splits=5)
rf_c = RandomForestClassifier(    
            n_estimators= 100,
            max_depth=5,
            ccp_alpha=1e-07,
         random_state=2020)
for tr_index,test_index in skfold.split(x_train,y_train):
    rf_c.fit(x_train[tr_index],y_train[tr_index])
    y_pred =  rf_c.predict(x_train)
    sf_rf_train_res_ensem+=y_pred
    sf_rf_test_res_ensem+=rf_c.predict(x_test)
sf_rf_train_res_ensem =np.round(sf_rf_train_res_ensem/5)
sf_rf_test_res_ensem =np.round(sf_rf_test_res_ensem/5)
#rf_c.fit(x_train,y_train)

피쳐가 복잡한 특성과 사기인가 아닌가 판별하는데 있어서 몇몇 피쳐들의 특성,
* 외국인이냐 아니냐
* 고위험 국가냐
* chbk의 횟수에 따른 사기인지 아닌지


등의 피쳐의 분포가 명확히 나눠져있어서 DT들을 엮어만든 뒤 앙상블한 rf모델이 좋은 성능을 발휘할 것이라 생각하고 채용했습니다.

In [ ]:
print("acc_train score : ", accuracy_score(sf_rf_train_res_ensem,y_train),",f1_score :",f1_score(sf_rf_train_res_ensem,y_train))
print("acc_train score : ", accuracy_score(sf_rf_test_res_ensem,y_test),",f1_score :",f1_score(sf_rf_test_res_ensem,y_test))


In [ ]:
model_performance_list.append(f1_score(sf_rf_test_res_ensem,y_test))

acc이 굉장이 높게 나왔는데 레이블의 분포가 불균형하기 때문에 사기가 아님 이라고만 답해도 85프로는 찍기때문에, f1스코어를 보는게 좋습니다 하지만 여기서도 f1스코어가 93프로를 달성했습니다!

In [ ]:
xgb_c = XGBClassifier(    
    learning_rate =0.005,
    n_estimators=100,
    max_depth=10,
    min_child_weight=8,
    gamma=0.0008,
    reg_alpha=1e-05,
    subsample=0.99,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=-1,
    scale_pos_weight=1,
    seed=2020)

피쳐가 복잡한 특성과 rf와 비슷하게 여러가지의 DT들을 엮어만든 뒤 가중치를 부과하여 앙상블한 모델로써 이역시도 좋은 성능을 발휘할 것이라 생각하고 채용했습니다

**light gbm도 써볼까 했지만 피쳐수가 적을때, lgbm은 오버피팅에 빠지기 쉽기 때문에 채용안했습니다.

In [ ]:
skfold = StratifiedKFold(n_splits=5)
sf_xgb_train_res_ensem = np.zeros(len(x_train))
sf_xgb_test_res_ensem =np.zeros(len(x_test))
for tr_index,test_index in skfold.split(x_train,y_train):
    xgb_c.fit(x_train[tr_index],y_train[tr_index])
    y_pred =  xgb_c.predict(x_train)
    sf_xgb_train_res_ensem+=y_pred
    sf_xgb_test_res_ensem+=xgb_c.predict(x_test)
sf_xgb_train_res_ensem =  np.round(sf_xgb_train_res_ensem/4)
sf_xgb_test_res_ensem = np.round(sf_xgb_test_res_ensem/4)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score
print("acc_train score : ", accuracy_score(sf_xgb_train_res_ensem,y_train),",f1_score :",f1_score(sf_xgb_train_res_ensem,y_train))
print("acc_train score : ", accuracy_score(sf_xgb_test_res_ensem,y_test),",f1_score :",f1_score(sf_xgb_test_res_ensem,y_test))


In [ ]:
model_performance_list.append(f1_score(sf_xgb_test_res_ensem,y_test))

네 어마어마한 성능을 발휘해줍니다.

# 성능 확인

In [ ]:
from pandas import Series

In [ ]:
plt.figure(figsize=(8, 8))
sns.barplot(model_list,model_performance_list)
plt.ylabel('performance')
plt.ylabel('pre-train model list')
plt.show()

이 두 모델의 예측결과를 앙상블 시켜보면 어떻게 될까요?

In [ ]:
print("rf+xgb ensemble f1_score :",f1_score(np.round((sf_xgb_test_res_ensem+sf_rf_test_res_ensem)/2),y_test))

In [ ]:
feature_importance = xgb_c.feature_importances_
Series_feat_imp = Series(feature_importance, index=df_train.drop(['isFradulent'],axis=1).columns)
plt.figure(figsize=(8, 8))
Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('xgboost Feature')
plt.show()

In [ ]:
Series_feat_imp = Series(feature_importance, index=df_train.drop(['isFradulent'],axis=1).columns)
plt.figure(figsize=(8, 8))
Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('Random forest Feature')
plt.show()

위 두 모델다 높은 성능을 발휘 해준만큼, 예측을 하는데 있어서 본 가중치를 높게 잡은 피쳐의 종류가 거의 동일하다는 점입니다. 차이점이라면 isForeignTransaction과 6-month-avg-chbk-amt가 되겠네요. 하지만 그마저도 근소한 차이로 1단계 순위정도 차이나니 feed이 굉장히 잘되었다고 봐도 될거같습니다. 

In [ ]:
from sklearn.metrics import roc_curve
fpr1, tpr1, thresholds1 = roc_curve(y_test, xgb_c.predict(x_test))
fpr2, tpr2, thresholds1 = roc_curve(y_test, rf_c.predict(x_test))

plt.plot(fpr1, tpr1, 'o-', ms=2, label="Logistic Regression")
plt.plot(fpr2, tpr2, 'o-', ms=2, label="Kernel SVM")
plt.legend()
plt.plot([0, 1], [0, 1], 'k--', label="random guess")
plt.xlabel('위양성률(Fall-Out)')
plt.ylabel('재현률(Recall)')
plt.title('ROC 커브')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,sf_xgb_test_res_ensem))